In [1]:
import music21 as m2
import glob
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.signal import windows, oaconvolve

In [7]:
import sys
from os import path

code_path = path.abspath(path.join('..','..','diss3_code','midi'))
print(code_path, path.exists(code_path))

sys.path.append(code_path)
import midi_utils as mu

/home/siegman/Documents/Shahar/BarIlan/Dissertation3/code/diss3_code/midi True


In [3]:
ngw = windows.gaussian(2001,200)
ngw = ngw/ngw.sum()

In [8]:
midi_paths = ['MIDI-Freestyle', 'MIDI-Interaction']
midi_paths = [path.abspath(path.join('..','..','..','data',p)) for p in midi_paths]
print([path.exists(p) for p in midi_paths])

midi_files = sum([glob.glob(path.join(mp,'*.mid')) for mp in midi_paths],[] )
midi_files[-5:]

[True, True]


['/home/siegman/Documents/Shahar/BarIlan/Dissertation3/data/MIDI-Interaction/Interaction_1025.mid',
 '/home/siegman/Documents/Shahar/BarIlan/Dissertation3/data/MIDI-Interaction/Interaction_1026.mid',
 '/home/siegman/Documents/Shahar/BarIlan/Dissertation3/data/MIDI-Interaction/Interaction_1031.mid',
 '/home/siegman/Documents/Shahar/BarIlan/Dissertation3/data/MIDI-Interaction/Interaction_1027.mid',
 '/home/siegman/Documents/Shahar/BarIlan/Dissertation3/data/MIDI-Interaction/Interaction_1046.mid']

In [42]:
def populate_music_dictionary(midi_files):
    all_ticks = {}
    for mf in midi_files:
        group_num = path.basename(mf).split('_')[1].split('.')[0]
        session = path.basename(mf).split('_')[0]
        group_ticks = mu.get_drumbeat_ticks_for_midi(mf)
        for track, ticks in group_ticks.items():
            print((group_num, session, track))
            assert (group_num, session, track) not in all_ticks
            all_ticks[(group_num, session, track)]= [t[0] for t in ticks if t[1]-t[0]>10] 
    return all_ticks

def get_signals_for_group_session(all_ticks,group,session,kernel):
    arrays = [to_dense(np.array(all_ticks[(group,session,par)])) for par in [43,45,48]]
    arrays = [oaconvolve(t,kernel) for t in arrays]
    la = min([len(a) for a in arrays])
    ret = np.zeros((3,la))
    for i in range(3):
        ret[i,:] =arrays[i][:la]
    return ret

def export_graphs_to_js_object(all_ticks, groups=None, kernel=ngw):
    data_obj = {}
    for k, v in all_ticks:
        gr = k[0]
        if groups is not None and gr not in groups:
            continue
        data_obj[gr] = data_obj.get(gr,{})
        array = np.round(oaconvolve(to_dense(np.array(v)),ngw)*100[::100], 3)
        
        ss = get_signals_for_group_session(all_ticks,'1009', 'Interaction' ,ngw)
        data_obj_k2 = {'Freestyle': 'FS', 'Interaction': 'IN'}[k[1]] 
        data_obj[gr][data_obj_k2][k[2]] = {'x': list(range(len(array))), 'y':list(array), 'type': 'scatter'}
    return data_obj
        
def get_signals_for_group_session(all_ticks,group,session,kernel):
    arrays = [to_dense(np.array(all_ticks[(group,session,par)])) for par in [43,45,48]]
    arrays = [oaconvolve(t,kernel) for t in arrays]
    la = min([len(a) for a in arrays])
    ret = np.zeros((3,la))
    for i in range(3):
        ret[i,:] =arrays[i][:la]
    return ret


def to_dense(t,max_len=None):
    max_len = max_len or t[-1]
    ret = np.zeros(max_len)
    max_index_to_include = np.sum(t<max_len).astype(int)
    ret[t[:max_index_to_include]]=1.
    return ret

def as_plotly_obj(all_ticks, group, session, n_points=100, plot_type='scatter'):
    kernel = ngw
    arrays = [to_dense(np.array(all_ticks[(group,session,par)])) for par in [43,45,48]]
    arrays = [oaconvolve(t,kernel) for t in arrays]
    shortest = min([len(a) for a in arrays])
    sampling_index = np.round(np.linspace(0,shortest-1,n_points)).astype(int)
    ret = [{
        "x": list(range(n_points)),
        "y": list(np.round(array[sampling_index],4)),
        "type": plot_type,
        "name": f"Participant {i}"
    } for i, array in enumerate(arrays,1)]
    return ret
    

In [10]:
all_ticks = populate_music_dictionary(midi_files)

('1009', 'Freestyle', 43)
('1009', 'Freestyle', 45)
('1009', 'Freestyle', 48)
('1025', 'Freestyle', 43)
('1025', 'Freestyle', 48)
('1025', 'Freestyle', 45)
('1021', 'Freestyle', 45)
('1021', 'Freestyle', 48)
('1021', 'Freestyle', 43)
('1018', 'Freestyle', 43)
('1018', 'Freestyle', 48)
('1018', 'Freestyle', 45)
('1008', 'Freestyle', 48)
('1008', 'Freestyle', 45)
('1008', 'Freestyle', 43)
('1048', 'Freestyle', 45)
('1048', 'Freestyle', 43)
('1048', 'Freestyle', 48)
('1026', 'Freestyle', 43)
('1026', 'Freestyle', 45)
('1026', 'Freestyle', 48)
('1024', 'Freestyle', 43)
('1024', 'Freestyle', 48)
('1024', 'Freestyle', 45)
('1023', 'Freestyle', 48)
('1023', 'Freestyle', 43)
('1023', 'Freestyle', 45)
('1033', 'Freestyle', 45)
('1033', 'Freestyle', 43)
('1033', 'Freestyle', 48)
('1045', 'Freestyle', 48)
('1045', 'Freestyle', 43)
('1045', 'Freestyle', 45)
('1034', 'Freestyle', 45)
('1034', 'Freestyle', 48)
('1034', 'Freestyle', 43)
('1046', 'Freestyle', 48)
('1046', 'Freestyle', 45)
('1046', 'Fr

In [11]:
all_ticks.__repr__()[:500]

"{('1009', 'Freestyle', 43): [1013, 1107, 1194, 1295, 1390, 2252, 2566, 2627, 2758, 2822, 2857, 2889, 2956, 3169, 3513, 3574, 3631, 3729, 3815, 3900, 4001, 4136, 5269, 5413, 6155, 6746, 7204, 7287, 7718, 7771, 8595, 8784, 9581, 9762, 9885, 10004, 10103, 10176, 10286, 10386, 10478, 10853, 10951, 11299, 12247, 12655, 12790, 12911, 13030, 13571, 13817, 13846, 13896, 13940, 13991, 14044, 14098, 14154, 14221, 14288, 14343, 14407, 14468, 14563, 14629, 14680, 14783, 14854, 14914, 14988, 15013, 15130, 15"

In [14]:
np.array([5.5]).astype(int)

array([5])

In [33]:
groups = ['1001', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1018', '1019', '1020', '1021', '1022', '1023', '1024', '1025', '1026', '1027',  '1030', '1031', '1032', '1033', '1034', '1035', '1036', '1037', '1038', '1039', '1040', '1041', '1042', '1043', '1044', '1045', '1046', '1047', '1048', '1049', '1050', '1051']

In [45]:
all_inter_data = {gr: as_plotly_obj(all_ticks, gr, 'Interaction', 250) for gr in groups}

In [46]:
import json

In [47]:
with open('session_data_interaction.json','wt') as a:
    json.dump(all_inter_data,a)

In [40]:
1/7

0.14285714285714285

In [41]:
np.round(1/7,2)

0.14